<a href="https://colab.research.google.com/github/VietHaDuong/gomoku-player/blob/main/gomokubyvhd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install git+https://github.com/sitfoxfly/gomoku-ai

  Cloning https://github.com/sitfoxfly/gomoku-ai to /tmp/pip-req-build-7sq8ynvh
  Running command git clone --filter=blob:none --quiet https://github.com/sitfoxfly/gomoku-ai /tmp/pip-req-build-7sq8ynvh
  Resolved https://github.com/sitfoxfly/gomoku-ai to commit 13efdaa4ad34f4c42fe600998528c6d2c09e63de
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
import re
import json
import random
from google.colab import userdata
from typing import Tuple
from gomoku import Agent
from gomoku.llm import OpenAIGomokuClient
from gomoku.core.models import Player, GameState, GameResult

class AgentG(Agent):

    def _setup(self):
      print("Setting up Garry (the player) ...")

      self.system_prompt = self._create_system_prompt()
      self.llm_client = OpenAIGomokuClient(api_key=userdata.get('omok'), model='Qwen/Qwen3-8B:featherless-ai', endpoint='https://router.huggingface.co/v1')

      print('Agent setup complete!')

    def _create_system_prompt(self) -> str:
        """Create the system prompt that teaches the LLM how to play Gomoku."""
        return """
        You are an expert Gomoku (Five-in-a-Row) player. Your goal is to get 5 of your pieces
          in a row (horizontally, vertically, or diagonally) while preventing your opponent from doing the same.

          Strategic priorities:
          1. WIN: If you can make 5 in a row, do it immediately
          2. BLOCK: If opponent can make 5 in a row, block them immediately
          3. CREATE THREATS: Build sequences of 2-3 pieces to create multiple winning opportunities
          4. CONTROL CENTER: The center area is most valuable for creating multiple directions
          5. CREATE FORKS: Try to create situations where you have multiple ways to win

          You must respond with valid JSON in this exact format:
          {
              "reasoning": "Brief explanation of your strategic thinking",
              "row": <row_number>,
              "col": <col_number>
          }

          The row and col must be valid coordinates (0-indexed). Choose only empty positions marked with '.'.
          """.strip()

    async def get_move(self, game_state: GameState) -> Tuple[int, int]:
        """Main method: Get the next move from our LLM."""
        print(f"\n🧠 {self.agent_id} is thinking...")

        try:

            board_str = game_state.format_board(formatter="standard")
            board_prompt = f"Current board state:\n{board_str}\n"
            board_prompt += f"Current player: {game_state.current_player.value}\n"

            # Create messages for the LLM
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": f"{board_prompt}\n\nPlease provide your next move as JSON."},
            ]

            print("💡 Full Prompt:\n\n")
            print(json.dumps(messages, indent=2, ensure_ascii=False))
            print()

            # Get response from LLM
            response = await self.llm_client.complete(messages)

            print("💡 Response:\n\n")
            print(response)
            print()

            if m := re.search(r"{[^}]+}", response, re.DOTALL):
                json_data = json.loads(m.group(0).strip())
                return json_data["row"], json_data["col"]

        except Exception as e:
            print(e)

        return self._get_fallback_move(game_state)

    def _get_fallback_move(self, game_state: GameState) -> Tuple[int, int]:
        """Simple fallback when LLM fails."""
        return random.choice(game_state.get_legal_moves())

print("🎉 Agent G is defined!")
print("   This agent demonstrates LLM-style strategic thinking.")

🎉 Agent G is defined!
   This agent demonstrates LLM-style strategic thinking.


In [12]:
from gomoku.agents import SimpleGomokuAgent
from gomoku.core.game_logic import GomokuGame
from gomoku.arena import GomokuArena
from gomoku.utils import ColorBoardFormatter

In [13]:
# Setup the competition arena
print("🏟️  Setting up the Competition Arena!")
print("=" * 50)

# Create arena with nice visualization
board_size = 8  # Smaller board for faster games
formatter = ColorBoardFormatter(board_size)
arena = GomokuArena(board_size=board_size, formatter=formatter, time_limit=10.0)

print(f"⚙️  Arena Configuration:")
print(f"   📏 Board size: {board_size}x{board_size}")
print(f"   ⏱️  Time limit: 10 seconds per move")
print(f"   🎨 Visualization: Color-coded board")

# Create our competing agents
agents = {
    'simple': SimpleGomokuAgent("SimpleBot"),
    'student_llm': AgentG("StudentAI")
}

print(f"\n🤖 Competitors Ready:")
for name, agent in agents.items():
    print(f"   • {agent.agent_id} ({name})")

print("\n🎮 Ready for competition!")

🏟️  Setting up the Competition Arena!
⚙️  Arena Configuration:
   📏 Board size: 8x8
   ⏱️  Time limit: 10 seconds per move
   🎨 Visualization: Color-coded board
Setting up Garry (the player) ...
Agent setup complete!

🤖 Competitors Ready:
   • SimpleBot (simple)
   • StudentAI (student_llm)

🎮 Ready for competition!


In [14]:
# Single game competition
print("🥊 COMPETITION: SimpleAgent vs AgentG")
print("=" * 60)

# Run the game
game_result = await arena.run_game(
    agents['simple'],
    agents['student_llm'],
    verbose=True
)

print("\n" + "=" * 60)
print("🏆 GAME RESULTS")
print("=" * 60)
print(f"🥇 Winner: {game_result['winner']}")
print(f"📊 Total moves: {game_result['moves']}")
print(f"⏱️  Game duration: {game_result.get('total_time', 0):.2f} seconds")
print(f"🎯 Victory condition: {game_result['reason']}")

if game_result.get('winning_sequence'):
    print(f"🏁 Winning sequence: {len(game_result['winning_sequence'])} positions")

print("\n🎮 Final Board with Winning Sequence:")
final_display = arena.draw_board_with_winning_sequence(
    game_result['final_board'],
    game_result.get('winning_sequence', [])
)
print(final_display)

🥊 COMPETITION: SimpleAgent vs AgentG

BLACK's turn (SimpleBot)
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 

Move: (4, 4) in 0.00s

WHITE's turn (StudentAI)
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  . 
 4  .  .  .  .  X  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 


🧠 StudentAI is thinking...
💡 Full Prompt:


[
  {
    "role": "system",
    "content": "You are an expert Gomoku (Five-in-a-Row) player. Your goal is to get 5 of your pieces\n          in a row (horizontally, vertically, or diagonally) while preventing your opponent from doing the same.\n\n          Strategic priorities:\n          1. WIN: If you can make 5 in a row, do it immedi